# NCAAM Network-Based Team Strength Analysis

## Objective
Implement network-based strength propagation for team tier classification:
- **Iterative Strength Ratings**: Massey/PageRank-style algorithm
- **Network-Weighted SOS**: Strength of schedule using network ratings
- **Multi-Hop Common Opponents**: 3-level transitive relationships

## NCAAM-Specific Characteristics
- 350+ teams, ~30-game season per team
- **Sparse graph** with **conference clusters**
- Cross-conference games create bridges between clusters
- Need deeper path exploration to connect distant teams
- Parameters: `max_hops=3`, `recency_decay=0.85`, `margin_cap=20`

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from datetime import datetime, timedelta
from collections import Counter

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

# NCAAM-specific config
CONFIG = {
    'sport': 'NCAAM',
    'max_hops': 3,          # Need deeper paths across conferences
    'recency_decay': 0.85,  # Early season less relevant
    'margin_cap': 20,       # Similar to NFL
    'iterations': 150,      # More iterations for larger network
    'tolerance': 0.0005     # Tighter tolerance for convergence
}

print(f"Config: {CONFIG}")

## Phase 1: Data Loading & Preparation

In [ ]:
# Load NCAAM data
data_file = Path().resolve().parent / 'data' / 'results' / 'ncaam_season_results.xlsx'
df = pd.read_excel(data_file)

print(f"Loaded {len(df)} NCAAM games")
print(f"Date range: {df['game_date'].min().date()} to {df['game_date'].max().date()}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
# Prepare game-level data
df['home_margin'] = df['home_score'] - df['away_score']
df['winner'] = np.where(df['home_margin'] > 0, df['home_team'], 
                        np.where(df['home_margin'] < 0, df['away_team'], None))
df['loser'] = np.where(df['home_margin'] > 0, df['away_team'], 
                       np.where(df['home_margin'] < 0, df['home_team'], None))
df['margin'] = df['home_margin'].abs()

games_with_result = df[df['winner'].notna()].copy()
print(f"Games with decisive result: {len(games_with_result)}")

# Get all teams
all_teams = set(df['home_team'].unique()) | set(df['away_team'].unique())
print(f"Total teams: {len(all_teams)}")

# Avg games per team
games_per_team = (len(df) * 2) / len(all_teams)
print(f"Avg games per team: {games_per_team:.1f}")

## Phase 2: Build Network Graph

NCAAM has a sparse graph with conference clusters. We'll analyze connectivity.

In [ ]:
def build_team_network(games_df, recency_decay=0.85):
    """Build weighted directed graph - optimized for sparse NCAAM network."""
    G = nx.DiGraph()
    
    for team in all_teams:
        G.add_node(team)
    
    games_sorted = games_df.sort_values('game_date')
    max_date = games_sorted['game_date'].max()
    
    edge_data = {}
    
    for _, game in games_sorted.iterrows():
        winner, loser = game['winner'], game['loser']
        margin = game['margin']
        
        days_ago = (max_date - game['game_date']).days
        recency_weight = recency_decay ** (days_ago / 7)
        
        key = (winner, loser)
        if key not in edge_data:
            edge_data[key] = {
                'games': 0,
                'total_margin': 0,
                'weighted_margin': 0,
                'total_weight': 0
            }
        
        edge_data[key]['games'] += 1
        edge_data[key]['total_margin'] += margin
        edge_data[key]['weighted_margin'] += margin * recency_weight
        edge_data[key]['total_weight'] += recency_weight
    
    for (winner, loser), data in edge_data.items():
        avg_margin = data['total_margin'] / data['games']
        weighted_avg = data['weighted_margin'] / data['total_weight']
        
        G.add_edge(winner, loser, 
                   games=data['games'],
                   avg_margin=avg_margin,
                   weighted_margin=weighted_avg)
    
    return G

G = build_team_network(games_with_result, CONFIG['recency_decay'])

print(f"Network Summary:")
print(f"  Nodes (teams): {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")
print(f"  Avg degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.1f}")
print(f"  Density: {nx.density(G):.4f}")

In [ ]:
# Analyze graph connectivity - critical for NCAAM
G_undirected = G.to_undirected()

# Find connected components
components = list(nx.connected_components(G_undirected))
print(f"\nConnectivity Analysis:")
print(f"  Connected components: {len(components)}")
print(f"  Largest component: {len(max(components, key=len))} teams")

if len(components) > 1:
    print(f"  Isolated teams: {sum(1 for c in components if len(c) == 1)}")
    small_components = [c for c in components if 1 < len(c) < 10]
    print(f"  Small clusters (2-9 teams): {len(small_components)}")

# Degree distribution
degrees = [d for _, d in G.degree()]
print(f"\nDegree Distribution:")
print(f"  Min: {min(degrees)}, Max: {max(degrees)}, Median: {np.median(degrees):.0f}")

# Low-connectivity teams (potential issues)
low_conn_teams = [n for n, d in G.degree() if d < 5]
print(f"  Teams with <5 connections: {len(low_conn_teams)}")

In [ ]:
# Detect conference-like clusters using community detection
try:
    from networkx.algorithms.community import louvain_communities
    communities = louvain_communities(G_undirected, seed=42)
    
    print(f"\nCommunity Detection (Conference Proxy):")
    print(f"  Communities found: {len(communities)}")
    
    # Size distribution
    sizes = sorted([len(c) for c in communities], reverse=True)
    print(f"  Largest 10 communities: {sizes[:10]}")
    
    # Store community membership
    team_community = {}
    for i, comm in enumerate(communities):
        for team in comm:
            team_community[team] = i
            
except ImportError:
    print("Community detection not available (needs networkx >= 2.6)")
    team_community = {team: 0 for team in all_teams}

In [ ]:
# Visualize (sample of top teams only due to size)
# Calculate simple win% for filtering
win_counts = dict(G.out_degree())
loss_counts = dict(G.in_degree())
win_pct = {team: win_counts.get(team, 0) / (win_counts.get(team, 0) + loss_counts.get(team, 0) + 0.001)
           for team in all_teams}

# Filter to top 50 teams by win%
top_teams = sorted(win_pct.items(), key=lambda x: -x[1])[:50]
top_team_names = [t[0] for t in top_teams]

G_sub = G.subgraph(top_team_names).copy()

fig, ax = plt.subplots(figsize=(14, 14))

node_colors = [win_pct.get(node, 0.5) for node in G_sub.nodes()]
pos = nx.spring_layout(G_sub, k=2, iterations=50, seed=42)

nodes = nx.draw_networkx_nodes(G_sub, pos, node_color=node_colors, cmap=plt.cm.RdYlGn,
                               node_size=400, alpha=0.8, vmin=0.3, vmax=0.9, ax=ax)
nx.draw_networkx_labels(G_sub, pos, font_size=6, ax=ax)
nx.draw_networkx_edges(G_sub, pos, alpha=0.15, arrows=True, arrowsize=8, ax=ax)

ax.set_title(f'NCAAM Top 50 Teams Network\n({G.number_of_nodes()} total teams)', fontsize=14)
fig.colorbar(nodes, ax=ax, label='Win %')
ax.axis('off')
plt.tight_layout()
plt.show()

## Phase 3: Iterative Strength Rating

For NCAAM's sparse graph, we need more iterations and careful handling of isolated teams.

In [ ]:
def compute_iterative_strength(games_df, max_iterations=150, tolerance=0.0005, margin_cap=20):
    """
    Compute network strength ratings - optimized for sparse NCAAM network.
    
    Uses smaller learning rate and more iterations for large sparse graph.
    """
    teams = set(games_df['home_team']) | set(games_df['away_team'])
    ratings = {team: 0.5 for team in teams}
    games = games_df[games_df['winner'].notna()].copy()
    
    # Smaller learning rate for stability with many teams
    learning_rate = 0.03
    
    history = []
    
    for iteration in range(max_iterations):
        new_ratings = ratings.copy()
        
        for _, game in games.iterrows():
            winner = game['winner']
            loser = game['loser']
            margin = min(game['margin'], margin_cap)
            
            winner_rating = ratings[winner]
            loser_rating = ratings[loser]
            
            total = winner_rating + loser_rating
            expected = winner_rating / total if total > 0 else 0.5
            
            surprise = 1 - expected
            adjustment = surprise * (margin / margin_cap) * learning_rate
            
            new_ratings[winner] += adjustment
            new_ratings[loser] -= adjustment
        
        # Normalize
        min_r = min(new_ratings.values())
        max_r = max(new_ratings.values())
        if max_r > min_r:
            new_ratings = {t: (r - min_r) / (max_r - min_r) for t, r in new_ratings.items()}
        
        max_change = max(abs(new_ratings[t] - ratings[t]) for t in teams)
        history.append(max_change)
        
        if max_change < tolerance:
            print(f"Converged at iteration {iteration + 1}")
            break
        
        ratings = new_ratings
    else:
        print(f"Did not converge in {max_iterations} iterations (final change: {max_change:.6f})")
    
    return ratings, history

# Compute ratings
network_ratings, convergence_history = compute_iterative_strength(
    df, 
    max_iterations=CONFIG['iterations'],
    tolerance=CONFIG['tolerance'],
    margin_cap=CONFIG['margin_cap']
)

# Top teams
ratings_df = pd.DataFrame([
    {'team': team, 'network_rating': rating}
    for team, rating in network_ratings.items()
]).sort_values('network_rating', ascending=False)

print("\nTop 15 Teams by Network Rating:")
print(ratings_df.head(15).to_string(index=False))

In [ ]:
# Plot convergence
plt.figure(figsize=(10, 4))
plt.plot(convergence_history)
plt.axhline(y=CONFIG['tolerance'], color='r', linestyle='--', label=f"Tolerance ({CONFIG['tolerance']})")
plt.xlabel('Iteration')
plt.ylabel('Max Rating Change')
plt.title('NCAAM: Convergence of Iterative Strength Algorithm')
plt.legend()
plt.yscale('log')
plt.tight_layout()
plt.show()

## Phase 4: Validate Ratings

In [ ]:
# Calculate simple team stats
team_stats = []
for team in all_teams:
    home_games = df[df['home_team'] == team]
    away_games = df[df['away_team'] == team]
    
    home_wins = (home_games['home_score'] > home_games['away_score']).sum()
    away_wins = (away_games['away_score'] > away_games['home_score']).sum()
    
    home_diff = (home_games['home_score'] - home_games['away_score']).sum()
    away_diff = (away_games['away_score'] - away_games['home_score']).sum()
    
    total_games = len(home_games) + len(away_games)
    
    opponents = list(home_games['away_team']) + list(away_games['home_team'])
    
    team_stats.append({
        'team': team,
        'games': total_games,
        'wins': home_wins + away_wins,
        'simple_win_pct': (home_wins + away_wins) / total_games if total_games > 0 else 0,
        'point_diff_avg': (home_diff + away_diff) / total_games if total_games > 0 else 0,
        'network_rating': network_ratings.get(team, 0.5),
        'community': team_community.get(team, -1),
        'opponents': opponents
    })

df_teams = pd.DataFrame(team_stats)

# Calculate SOS
win_pct_map = df_teams.set_index('team')['simple_win_pct'].to_dict()
df_teams['simple_sos'] = df_teams['opponents'].apply(
    lambda opps: np.mean([win_pct_map.get(o, 0.5) for o in opps]) if opps else 0.5
)
df_teams['network_sos'] = df_teams['opponents'].apply(
    lambda opps: np.mean([network_ratings.get(o, 0.5) for o in opps]) if opps else 0.5
)

df_teams = df_teams.drop('opponents', axis=1).sort_values('network_rating', ascending=False)
print(f"Team stats calculated for {len(df_teams)} teams")
df_teams.head(10)

In [ ]:
# Correlation analysis
# Filter to teams with enough games for reliable stats
df_reliable = df_teams[df_teams['games'] >= 15].copy()
print(f"Teams with 15+ games: {len(df_reliable)}")

corr_win_pct = df_reliable['network_rating'].corr(df_reliable['simple_win_pct'])
corr_point_diff = df_reliable['network_rating'].corr(df_reliable['point_diff_avg'])
corr_sos = df_reliable['network_sos'].corr(df_reliable['simple_sos'])

print(f"\nCorrelation Analysis (teams with 15+ games):")
print(f"  Network Rating vs Win%: {corr_win_pct:.3f}")
print(f"  Network Rating vs Point Diff: {corr_point_diff:.3f}")
print(f"  Network SOS vs Simple SOS: {corr_sos:.3f}")

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].scatter(df_reliable['simple_win_pct'], df_reliable['network_rating'], alpha=0.5, s=20)
axes[0].set_xlabel('Simple Win %')
axes[0].set_ylabel('Network Rating')
axes[0].set_title(f'Network vs Win % (r={corr_win_pct:.3f})')

axes[1].scatter(df_reliable['point_diff_avg'], df_reliable['network_rating'], alpha=0.5, s=20)
axes[1].set_xlabel('Avg Point Differential')
axes[1].set_ylabel('Network Rating')
axes[1].set_title(f'Network vs Point Diff (r={corr_point_diff:.3f})')

axes[2].scatter(df_reliable['simple_sos'], df_reliable['network_sos'], alpha=0.5, s=20)
axes[2].set_xlabel('Simple SOS')
axes[2].set_ylabel('Network SOS')
axes[2].set_title(f'Network SOS vs Simple SOS (r={corr_sos:.3f})')

plt.tight_layout()
plt.show()

## Phase 5: Network-Based Tier Classification

In [ ]:
# Classify tiers (using reliable teams only)
q75_simple, q25_simple = df_reliable['simple_win_pct'].quantile([0.75, 0.25])
q75_network, q25_network = df_reliable['network_rating'].quantile([0.75, 0.25])

df_teams['simple_tier'] = df_teams['simple_win_pct'].apply(
    lambda x: 'Elite' if x >= q75_simple else ('Bottom' if x <= q25_simple else 'Mid')
)
df_teams['network_tier'] = df_teams['network_rating'].apply(
    lambda x: 'Elite' if x >= q75_network else ('Bottom' if x <= q25_network else 'Mid')
)

print("Tier Classification:")
print(f"  Simple thresholds: Elite >= {q75_simple:.3f}, Bottom <= {q25_simple:.3f}")
print(f"  Network thresholds: Elite >= {q75_network:.3f}, Bottom <= {q25_network:.3f}")

# Agreement
df_compare = df_teams[df_teams['games'] >= 15]
agreement = pd.crosstab(df_compare['simple_tier'], df_compare['network_tier'], margins=True)
print(f"\nTier Agreement (teams with 15+ games):")
print(agreement)

# Significant disagreements
disagreements = df_compare[
    (df_compare['simple_tier'] == 'Elite') & (df_compare['network_tier'] != 'Elite') |
    (df_compare['simple_tier'] != 'Elite') & (df_compare['network_tier'] == 'Elite')
]
print(f"\nElite tier disagreements ({len(disagreements)}):")
print(disagreements[['team', 'games', 'simple_win_pct', 'network_rating', 
                     'simple_sos', 'network_sos', 'simple_tier', 'network_tier']].head(10).to_string(index=False))

## Phase 6: Multi-Hop Common Opponent Analysis

For NCAAM, multi-hop paths are critical for connecting teams across conferences.

In [ ]:
def find_common_opponent_paths(G, team_a, team_b, max_hops=3):
    """Find paths through common opponents - optimized for sparse graphs."""
    G_undirected = G.to_undirected()
    
    if team_a not in G or team_b not in G:
        return []
    
    # Check if teams are in same component
    if not nx.has_path(G_undirected, team_a, team_b):
        return []
    
    paths = []
    try:
        # Limit paths to avoid combinatorial explosion
        path_gen = nx.all_simple_paths(G_undirected, team_a, team_b, cutoff=max_hops + 1)
        for i, path in enumerate(path_gen):
            if len(path) > 2:
                paths.append(path)
            if i > 1000:  # Limit for performance
                break
    except nx.NetworkXNoPath:
        pass
    
    return paths

def evaluate_matchup_via_paths(G, team_a, team_b, ratings, max_hops=3):
    """Evaluate teams through common opponent network."""
    paths = find_common_opponent_paths(G, team_a, team_b, max_hops)
    
    if not paths:
        return None, 0, []
    
    a_total_score = 0
    b_total_score = 0
    total_weight = 0
    path_details = []
    
    for path in paths:
        intermediates = path[1:-1]
        a_path_score = 0
        b_path_score = 0
        path_weight = 0
        
        for intermediate in intermediates:
            int_rating = ratings.get(intermediate, 0.5)
            
            if G.has_edge(team_a, intermediate):
                margin = G[team_a][intermediate].get('avg_margin', 0)
                a_path_score += margin * int_rating
            elif G.has_edge(intermediate, team_a):
                margin = G[intermediate][team_a].get('avg_margin', 0)
                a_path_score -= margin * int_rating
            
            if G.has_edge(team_b, intermediate):
                margin = G[team_b][intermediate].get('avg_margin', 0)
                b_path_score += margin * int_rating
            elif G.has_edge(intermediate, team_b):
                margin = G[intermediate][team_b].get('avg_margin', 0)
                b_path_score -= margin * int_rating
            
            path_weight += int_rating
        
        if path_weight > 0:
            a_total_score += a_path_score
            b_total_score += b_path_score
            total_weight += path_weight
            
            path_details.append({
                'path': ' -> '.join(path),
                'hops': len(path) - 2,
                'a_score': a_path_score / path_weight if path_weight else 0,
                'b_score': b_path_score / path_weight if path_weight else 0
            })
    
    if total_weight == 0:
        return None, 0, []
    
    edge = (a_total_score - b_total_score) / total_weight
    return edge, len(paths), path_details

# Example: Compare top team vs a mid-tier team
top_team = df_teams.iloc[0]['team']
mid_team = df_teams[df_teams['network_tier'] == 'Mid'].iloc[0]['team']

edge, num_paths, details = evaluate_matchup_via_paths(G, top_team, mid_team, network_ratings, CONFIG['max_hops'])

print(f"Matchup Analysis: {top_team} vs {mid_team}")
print(f"Paths found: {num_paths}")
if edge is not None:
    print(f"Common opponent edge: {edge:.2f} ({'favors ' + top_team if edge > 0 else 'favors ' + mid_team})")
    
    if details:
        path_df = pd.DataFrame(details)
        print(f"\nPaths by hop count:")
        print(path_df.groupby('hops').size())
else:
    print("No paths found - teams may be in different components")

In [ ]:
# Analyze cross-conference connectivity
# Find paths between teams in different communities

def analyze_cross_conference_paths(G, team_community, ratings, sample_size=20):
    """Analyze path lengths between teams in different communities."""
    results = []
    
    # Sample teams from different communities
    comm_teams = {}
    for team, comm in team_community.items():
        if comm not in comm_teams:
            comm_teams[comm] = []
        comm_teams[comm].append(team)
    
    # Get top teams from different large communities
    large_comms = [c for c, teams in comm_teams.items() if len(teams) >= 10][:5]
    
    if len(large_comms) < 2:
        print("Not enough large communities for cross-conference analysis")
        return None
    
    G_und = G.to_undirected()
    
    for i, comm1 in enumerate(large_comms):
        for comm2 in large_comms[i+1:]:
            # Sample teams from each community
            teams1 = comm_teams[comm1][:3]
            teams2 = comm_teams[comm2][:3]
            
            for t1 in teams1:
                for t2 in teams2:
                    if t1 in G and t2 in G:
                        try:
                            path_len = nx.shortest_path_length(G_und, t1, t2)
                            results.append({
                                'comm1': comm1,
                                'comm2': comm2,
                                'team1': t1,
                                'team2': t2,
                                'path_length': path_len
                            })
                        except nx.NetworkXNoPath:
                            results.append({
                                'comm1': comm1,
                                'comm2': comm2,
                                'team1': t1,
                                'team2': t2,
                                'path_length': -1  # No path
                            })
    
    return pd.DataFrame(results)

cross_conf_df = analyze_cross_conference_paths(G, team_community, network_ratings)
if cross_conf_df is not None and len(cross_conf_df) > 0:
    valid_paths = cross_conf_df[cross_conf_df['path_length'] > 0]
    print(f"\nCross-Conference Path Analysis:")
    print(f"  Pairs analyzed: {len(cross_conf_df)}")
    print(f"  Pairs with paths: {len(valid_paths)}")
    if len(valid_paths) > 0:
        print(f"  Avg path length: {valid_paths['path_length'].mean():.1f}")
        print(f"  Path length distribution:")
        print(valid_paths['path_length'].value_counts().sort_index())

## Phase 7: Coverage Analysis by Network Tier

In [ ]:
# Calculate handicap coverage
handicaps = [0, 5, 10, 11, 13, 15]

for h in handicaps:
    df[f'home_covers_{h}pt'] = (df['spread_result_difference'] + h) >= 0
    df[f'away_covers_{h}pt'] = df['spread_result_difference'] <= h

# Analyze coverage (use teams with 15+ games)
reliable_teams = df_teams[df_teams['games'] >= 15]['team'].tolist()
results = []

for h in handicaps:
    for tier_type in ['simple_tier', 'network_tier']:
        for tier in ['Elite', 'Mid', 'Bottom']:
            tier_teams = df_teams[(df_teams[tier_type] == tier) & 
                                  (df_teams['team'].isin(reliable_teams))]['team'].tolist()
            
            total_games = 0
            total_covers = 0
            
            for team in tier_teams:
                hg = df[df['home_team'] == team]
                total_games += len(hg)
                total_covers += hg[f'home_covers_{h}pt'].sum()
                
                ag = df[df['away_team'] == team]
                total_games += len(ag)
                total_covers += ag[f'away_covers_{h}pt'].sum()
            
            results.append({
                'handicap': h,
                'tier_type': tier_type,
                'tier': tier,
                'games': total_games,
                'covers': total_covers,
                'cover_pct': total_covers / total_games if total_games > 0 else 0
            })

df_coverage = pd.DataFrame(results)

print("Coverage % by Tier Type and Handicap:")
print("="*60)

for tier_type in ['simple_tier', 'network_tier']:
    print(f"\n{tier_type.replace('_', ' ').title()}:")
    pivot = df_coverage[df_coverage['tier_type'] == tier_type].pivot(
        index='handicap', columns='tier', values='cover_pct'
    )[['Elite', 'Mid', 'Bottom']]
    print((pivot * 100).round(1))

In [ ]:
# Plot coverage comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'Elite': '#2ecc71', 'Mid': '#f39c12', 'Bottom': '#e74c3c'}

for idx, tier_type in enumerate(['simple_tier', 'network_tier']):
    ax = axes[idx]
    
    for tier in ['Elite', 'Mid', 'Bottom']:
        data = df_coverage[(df_coverage['tier_type'] == tier_type) & (df_coverage['tier'] == tier)]
        ax.plot(data['handicap'], data['cover_pct'] * 100, 
                marker='o', label=tier, color=colors[tier], linewidth=2)
    
    ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    ax.set_xlabel('Handicap Points')
    ax.set_ylabel('Coverage %')
    ax.set_title(f'{tier_type.replace("_", " ").title()}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('NCAAM: Coverage by Tier Classification Method', fontsize=14)
plt.tight_layout()
plt.show()

## Summary & Recommendations

In [ ]:
print("="*70)
print("NCAAM NETWORK STRENGTH ANALYSIS: SUMMARY")
print("="*70)

print(f"\n1. DATA")
print(f"   - Games analyzed: {len(df)}")
print(f"   - Teams: {len(all_teams)}")
print(f"   - Network edges: {G.number_of_edges()}")
print(f"   - Graph density: {nx.density(G):.4f}")
print(f"   - Connected components: {len(components)}")

print(f"\n2. NETWORK RATING VALIDATION (teams with 15+ games)")
print(f"   - Correlation with Win%: {corr_win_pct:.3f}")
print(f"   - Correlation with Point Diff: {corr_point_diff:.3f}")
print(f"   - Network SOS vs Simple SOS: {corr_sos:.3f}")

print(f"\n3. TIER CLASSIFICATION")
tier_agreement = (df_compare['simple_tier'] == df_compare['network_tier']).mean() * 100
print(f"   - Agreement rate: {tier_agreement:.1f}%")
print(f"   - Elite disagreements: {len(disagreements)} teams")

print(f"\n4. MULTI-HOP ANALYSIS")
print(f"   - Max hops: {CONFIG['max_hops']}")
print(f"   - Critical for cross-conference comparisons")

print(f"\n5. COVERAGE BY TIER (10pt handicap)")
for tier_type in ['simple_tier', 'network_tier']:
    print(f"   {tier_type.replace('_', ' ').title()}:")
    for tier in ['Elite', 'Mid', 'Bottom']:
        row = df_coverage[(df_coverage['tier_type'] == tier_type) & 
                         (df_coverage['tier'] == tier) &
                         (df_coverage['handicap'] == 10)]
        if len(row) > 0:
            pct = row['cover_pct'].values[0] * 100
            print(f"      {tier}: {pct:.1f}%")

print(f"\n6. NCAAM-SPECIFIC FINDINGS")
print(f"   - Sparse graph requires network analysis for cross-conf comparisons")
print(f"   - Community detection identifies conference-like clusters")
print(f"   - Network rating helps identify overrated/underrated teams")

In [ ]:
# Export team data
export_df = df_teams[['team', 'games', 'wins', 'simple_win_pct', 'network_rating', 
                      'point_diff_avg', 'simple_sos', 'network_sos', 'community',
                      'simple_tier', 'network_tier']].copy()

export_file = Path().resolve().parent / 'data' / 'results' / 'ncaam_network_ratings.csv'
export_df.to_csv(export_file, index=False)
print(f"Exported team ratings to: {export_file}")

export_df.head(15)